In [ ]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os
import numpy as np
import sys 

sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import matplotlib.pyplot as plt

In [ ]:
# Define the path to the folder containing the videos and other files 
path_to_video_folder: str = "/Users/zacharykelly/Desktop/geoff_videos_blnk_folder/HERO_gka/videos"

# Generate paths to all of the videos in said folder 
video_paths: list[str] = [os.path.join(path_to_video_folder, filename)
                          for filename in os.listdir(path_to_video_folder)
                          if '.avi' in filename
                         ]

In [ ]:
# Generate output folder 
output_folder_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_analysis/PuffLight/modulate/HERO_gka/videos"
if(not os.path.exists(output_folder_path)):
    os.mkdir(output_folder_path)

In [ ]:
# Write the prediction function
def predict_eye_features(args: tuple) -> None:
    # Extract the arguments from the arguments tuple 
    filepath, crop_box, target_size, temp_dir_path, output_folder_path = args
    threshold_value: int = 225

    # Define the portion of the video to crop out 
    t, b, l, r = crop_box

    # Extract the name of this video
    video_name: str = os.path.splitext(os.path.basename(filepath.rstrip('/')))[0]

    # Find the FPS of the video 
    video_fps: float = Pi_util.inspect_video_FPS(filepath)

    # Videos are small, so we can load them entirely in from memory 
    video_as_arr: np.ndarray = Pi_util.destruct_video(filepath, is_grayscale=True)

    # Crop out only the eye from the video
    # and set the rest of the frame to black 
    video_cropped: np.ndarray = video_as_arr[:, t:b, l:r].copy()
    white_pixels: np.ndarray = np.mean(video_cropped, axis=(0,))  > threshold_value
    video_cropped[:, white_pixels] = 0

    # Resize the video to not a small resolution 
    y_offset = (target_size[0] - video_cropped.shape[1]) // 2
    x_offset = (target_size[1] - video_cropped.shape[2]) // 2

    video_resized: np.ndarray = np.zeros((len(video_cropped), *target_size), dtype=np.uint8)
    video_resized[:, y_offset:y_offset + video_cropped.shape[1], x_offset:x_offset + video_cropped.shape[2]] = video_cropped

    # Generate a temp video from this cropped video 
    temp_video_path: str = os.path.join(temp_dir_path, f"temp_{video_name}.avi")
    Pi_util.frames_to_video(video_resized, temp_video_path, video_fps)

    # Extract the eye features for this video
    eye_features: list[dict] = extract_eye_features.extract_eye_features(temp_video_path, is_grayscale=True, visualize_results=False, pupil_feature_method='pylids', safe_execution=True)

    # Save the features 
    eye_features_dict: dict = {}
    eye_features_dict["eye_features"] = eye_features
    eye_features_dict["metadata"] = {'threshold_value': {'v': threshold_value, 
                                                    'desc': "binary mask constructed from avg cropped frame. Pixels above this value=0. Done to remove light around the eye"
                                                   },
                                'crop_box': {'v': crop_box, 
                                             'desc': "box cropped out from original video to isolate the eye (t, b, l, r)"
                                            },
                                'target_size': {'v': target_size,
                                                'desc': "target size after crop + threshold. Eye video is centered via padding to reach this size"},
                                'model_names': {'v': ('pytorch-pupil-v1', 'pytorch-eyelid-v1'), 
                                                'desc': "models used for pupil/eyelid fitting"
                                               }
                               }
    
    
    
    scipy.io.savemat(os.path.join(output_folder_path, f"{video_name}_eye_features.mat"), 
                    {"eye_features": eye_features_dict}
                    )
    
    # Remvove the temp avi video 
    os.remove(temp_video_path)


In [ ]:
# Generate a temp output directory 
temp_dir_path: str = './temp_blnk_pipeline'
if(not os.path.exists(temp_dir_path)):
    os.mkdir(temp_dir_path)

# Define the crop box and target size 
t, b, l, r = 140, 275, 190, 425
crop_box: tuple = (t, b, l, r)
target_size: tuple = (480, 640)

# Define the argument list that each function 
# will get 
args_list: list[list] = [ (filepath, crop_box, target_size, temp_dir_path, output_folder_path)
                          for filepath in video_paths 
                        ]

# Process the videos 
for video_num, video_args in enumerate(args_list):
    filepath: str = video_args[0]
    if(not any(f"00{trial}" in filepath for trial in range(5, 10))): continue

    print(f"Processing: {video_num}/{len(args_list)} | filepath: {filepath}", flush=True)
    predict_eye_features(video_args)
    

Processing: 4/58
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_HERO_gka_modulatedirection-Mel_phase-3.14_trial-005_side-R.avi
Video metadata: 
  Overall # of frames:    11160
  Duration of video [s]:  62.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


  2%|▏         | 263/11160 [00:08<05:55, 30.65it/s]


KeyboardInterrupt: 